In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("borhanitrash/animal-image-classification-dataset")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/animal-image-classification-dataset


In [25]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator


# Image dimensions and batch size
img_width, img_height = 150, 150
batch_size = 32

# Data augmentation and preprocessing (same as before)
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    '/kaggle/input/animal-image-classification-dataset/Animals',
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training')

validation_generator = train_datagen.flow_from_directory(
    '/kaggle/input/animal-image-classification-dataset/Animals',
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation')

# Load pre-trained VGG16 model (without the classification layers)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

# Freeze the base model's layers 
for layer in base_model.layers:
    layer.trainable = False

# Add new classification layers on top
x = base_model.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)  # Add a dense layer
predictions = Dense(train_generator.num_classes, activation='softmax')(x) # Output layer

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model (adjust epochs as needed)
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size)

# Save the model
model.save("animal_classifier_vgg16.h5")

Found 2400 images belonging to 3 classes.
Found 600 images belonging to 3 classes.
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 28s 227ms/step - accuracy: 0.7175 - loss: 0.8762 - val_accuracy: 0.8663 - val_loss: 0.3449
Epoch 2/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 8s 103ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.8750 - val_loss: 0.3187
Epoch 3/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 18s 226ms/step - accuracy: 0.8865 - loss: 0.2618 - val_accuracy: 0.8906 - val_loss: 0.2966
Epoch 4/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.8750 - val_loss: 0.3726
Epoch 5/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 18s 217ms/step - accuracy: 0.9260 - loss: 0.1766 - val_accuracy: 0.8906 - val_loss: 0.3164
Epoch 6/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.9167 - val_loss: 0.1654
Epoch 7/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 17s 212ms/step - accuracy: 0.9006 - l

In [30]:
import numpy as np
from tensorflow.keras.preprocessing import image
import tensorflow as tf
import os

# Load the saved model
model = tf.keras.models.load_model('/kaggle/working/animal_classifier_vgg16.h5')

# Function to preprocess a single image
def preprocess_image(image_path, img_width=150, img_height=150):
    img = image.load_img(image_path, target_size=(img_width, img_height))  # Resize image
    img_array = image.img_to_array(img)  # Convert image to numpy array
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.  # Rescale image
    return img_array

# Function to predict the class of a given image
def predict_image_class(image_path):
    img_array = preprocess_image(image_path)
    predictions = model.predict(img_array)  # Get prediction
    predicted_class = np.argmax(predictions, axis=-1)  # Get the class index with highest probability
    return predicted_class[0], predictions  # Return the predicted class index and probabilities

# Example usage:
image_directory = '/kaggle/input/catsnake'  # Replace with your image directory
image_files = os.listdir(image_directory)
image_path = os.path.join(image_directory, image_files[0])  # Use the first image in the directory

print(f"Using image path: {image_path}")

# Predict the class
predicted_class, predictions = predict_image_class(image_path)

# Get the class names from the training generator
class_names = list(train_generator.class_indices.keys())  # List of class names (e.g., 'cat', 'dog', 'snake')

# Print the result
print(f"Predicted class index: {predicted_class}")
print(f"Predicted class name: {class_names[predicted_class]}")
print(f"Prediction probabilities: {predictions}")


Using image path: /kaggle/input/catsnake/snake.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 521ms/step
Predicted class index: 2
Predicted class name: snakes
Prediction probabilities: [[8.677103e-09 5.569593e-10 1.000000e+00]]


In [14]:
list(train_generator.class_indices.keys())

['cats', 'dogs', 'snakes']